In [2]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset


device = "cuda:5" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "whisper-large-v3-turbo"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
)


In [3]:
ds = load_dataset("mozilla-foundation/common_voice_16_1", "ko")

In [ ]:
ds

In [ ]:
results = pipe(ds["test"]['path'])

In [4]:
import jiwer

hyp = [list(d.values())[0].strip() for d in results]
ref = ds["test"]['sentence']

NameError: name 'results' is not defined

In [ ]:
hyp

In [40]:
wer = jiwer.wer(ref, hyp)
cer = jiwer.cer(ref, hyp)

In [ ]:
print(wer, cer)

In [5]:
from datasets import Audio
import jiwer

def prepare_dataset(batch):
    audio = batch['audio']
    batch["length"] = float(audio["array"].size / audio["sampling_rate"])
    #batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    #batch["labels"] = tokenizer(batch["text"]).input_ids
    return batch

ds_test = ds['test'].map(prepare_dataset)

In [9]:
len = 0.0
for l in ds_test['length']:
    len += float(l)

print(len)

1473.839999999999


In [ ]:
# 122.7초

